In [1]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!pip install transformers
from transformers import pipeline
senti = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.1 MB/s eta 0:00:00


In [3]:
import tweepy

In [4]:
api_key = "Zf5TkFjjbqxyzC02cFNbrfd2V"
api_key_secret = "afUdvsB5rueFfZhAFguKlpbhzyFwL1rEp898MSj1HBBh7In8Sk"

access_token = '1420374454392156165-XUOMDIw2YmAIcF1xZVyp4Y50JEGUlB'
access_token_secret = 'rVOG17Kl7vZf8wB3cLt9X6YRk3LrEJctL48KyqX2SPqJI'

In [5]:
keyword = 'NASDAQ stocks'

In [6]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

In [7]:
api = tweepy.API(auth)

In [167]:
# public_tweets = api.search_tweets(q=keyword, tweet_mode='extended', count=1000)
cursor = tweepy.Cursor(api.search_tweets, q=keyword, count=200,tweet_mode='extended').items(1000)

In [168]:
columns = ['Time', 'User', 'Tweet']
data = []

In [169]:
# for tweet in public_tweets:
#     data.append([tweet.created_at, tweet.user.screen_name, tweet.full_text])

In [170]:
for tweet in cursor:
    data.append([tweet.created_at, tweet.user.screen_name, tweet.full_text])

In [171]:
df = pd.DataFrame(data, columns=columns)
df.to_csv('tweets.csv')

# NLP

In [172]:
english_stopwords = stopwords.words('english')

In [173]:
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in english_stopwords]

In [174]:
stemmer = SnowballStemmer(language='english')

In [175]:
def tokenize(text):
    return [stemmer.stem(word) for word in word_tokenize(text)]

In [176]:
small_vect = CountVectorizer()
vectorizer = CountVectorizer(lowercase=True, 
                             tokenizer=tokenize,
                             stop_words=english_stopwords,
                             max_features=1000)

In [177]:
df = pd.read_csv('tweets.csv')

In [178]:
%%time
df['Result'] = df['Tweet'].apply(lambda x: senti(x))
df['sentiment'] = df['Result'].apply(lambda x: (x[0]['label']))

CPU times: user 14min 23s, sys: 1.12 s, total: 14min 24s
Wall time: 14min 28s


In [179]:
df['sentiment'].value_counts(normalize=True)

POSITIVE    0.885
NEGATIVE    0.115
Name: sentiment, dtype: float64

In [180]:
import plotly.express as px
fig = px.histogram(df, x='sentiment',color='sentiment',text_auto=True)
fig.show()

In [181]:
df.to_csv('tweets_sentiments.csv')

In [182]:
from sklearn.linear_model import LogisticRegression

In [183]:
MAX_ITER = 1000
model = LogisticRegression(max_iter=MAX_ITER, solver='sag')

In [184]:
train_df = pd.read_csv('tweets_sentiments.csv')

In [185]:
train_inputs = train_df['Tweet']
train_targets = train_df['sentiment']

In [186]:
# pd.DataFrame(train_inputs)
train_inputs

0      RT @Schuldensuehner: Yields up, Tech stocks do...
1      Make $1000/Day with  fabulous  📈  $VET  📏  #NA...
2      $BNGO STRONG BUY \nBased on 5 analysts offerin...
3      RT @agent_00wo: I trash the 1-percentile Bears...
4      RT @PrimeXBT: 📉 The #Nasdaq is experiencing a ...
                             ...                        
995    $KBH  new alert at https://t.co/A7qrDarJHY  #s...
996    $QQQ  new alert at https://t.co/A7qrDarJHY  #s...
997    RT @Vltra_MK: Why won't [they] ever cover this...
998    EXPLAINER-How are money market funds preparing...
999    #NASDAQ E-mini #Futures #NQ_F ⬆️88.75 Points (...
Name: Tweet, Length: 1000, dtype: object

In [187]:
%%time
vectorizer.fit(train_inputs)
# vectorizer.get_feature_names_out()
train_inputs = vectorizer.transform(train_inputs)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'s", 'abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'might', 'must', "n't", 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'veri', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.



CPU times: user 869 ms, sys: 2 ms, total: 871 ms
Wall time: 876 ms


In [188]:
%%time
model.fit(train_inputs, train_targets)

CPU times: user 188 ms, sys: 4 µs, total: 188 ms
Wall time: 190 ms


LogisticRegression(max_iter=1000, solver='sag')

In [189]:
train_preds = model.predict(train_inputs)
train_targets

0      NEGATIVE
1      POSITIVE
2      POSITIVE
3      NEGATIVE
4      POSITIVE
         ...   
995    POSITIVE
996    POSITIVE
997    NEGATIVE
998    NEGATIVE
999    POSITIVE
Name: sentiment, Length: 1000, dtype: object

In [190]:
pd.Series(train_preds).value_counts()

POSITIVE    896
NEGATIVE    104
dtype: int64

In [191]:
pd.Series(train_targets).value_counts()

POSITIVE    885
NEGATIVE    115
Name: sentiment, dtype: int64

In [192]:
from sklearn.metrics import accuracy_score

In [193]:
accuracy_score(train_targets, train_preds)

0.985

In [277]:
prv = pd.Series(train_targets).value_counts()[0]

## Working on 5000 rows Dataset

In [278]:
df2 = pd.read_csv('tweets_2.csv')

In [279]:
inputs = df2.iloc[:1000, 3:4]
inputs = inputs['Tweet']

In [280]:
inputs

0      US STOCKS-Indexes jump on inflation data; Nasd...
1      S&amp;P &amp; NasDaq Index stocks with Low RSI...
2      The Nasdaq was set to post its biggest quarter...
3      2023-03-31\nhttps://t.co/cBypAMtNn1\n\nNasdaq ...
4      NEWS: $CKPT Checkpoint Therapeutics Announces ...
                             ...                        
995    $SPY Breaking News: Stocks close lower, Nasdaq...
996    $BRQS shorts still have the upper hand. When t...
997    Updated version! Most likely path for S&amp;P5...
998    $icnm If anyone ever doubted this company for ...
999    SELL TESLA INC, NASDAQ: TSLA, $191.81 #investi...
Name: Tweet, Length: 1000, dtype: object

In [281]:
%%time
vectorizer.fit(inputs)
# vectorizer.get_feature_names_out()
inputs = vectorizer.transform(inputs)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



CPU times: user 1.36 s, sys: 1.02 ms, total: 1.36 s
Wall time: 1.36 s


In [282]:
train_preds = model.predict(inputs)

In [283]:
new1 = pd.Series(train_preds).value_counts()[0]
pd.Series(train_preds).value_counts()

POSITIVE    853
NEGATIVE    147
dtype: int64

In [284]:
percentage = ((new1-prv)/new1)*100

In [285]:
if percentage > 0:
  print("increase of",percentage)
else:
  print("decrease of",abs(percentage))

decrease of 3.751465416178194


## 2nd DF

In [286]:
inputs = df2.iloc[1001:2000, 3:4]
inputs = inputs['Tweet']

In [287]:
%%time
vectorizer.fit(inputs)
# vectorizer.get_feature_names_out()
inputs = vectorizer.transform(inputs)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



CPU times: user 1.92 s, sys: 3.97 ms, total: 1.93 s
Wall time: 1.98 s


In [288]:
train_preds = model.predict(inputs)

In [289]:
new2 = pd.Series(train_preds).value_counts()[0]
pd.Series(train_preds).value_counts()

POSITIVE    837
NEGATIVE    162
dtype: int64

In [290]:
percentage = ((new2-new1)/new2)*100

In [291]:
if percentage > 0:
  print("increase of",percentage)
else:
  print("decrease of",abs(percentage))

decrease of 1.911589008363202


## 3rd DF

In [292]:
inputs = df2.iloc[2001:3000, 3:4]
inputs = inputs['Tweet']

In [293]:
%%time
vectorizer.fit(inputs)
# vectorizer.get_feature_names_out()
inputs = vectorizer.transform(inputs)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



CPU times: user 2.58 s, sys: 2.94 ms, total: 2.58 s
Wall time: 2.65 s


In [294]:
train_preds = model.predict(inputs)
new3 = pd.Series(train_preds).value_counts()[0]
pd.Series(train_preds).value_counts()

POSITIVE    901
NEGATIVE     98
dtype: int64

In [295]:
percentage = ((new3-new2)/new3)*100

In [296]:
if percentage > 0:
  print("increase of",percentage)
else:
  print("decrease of",abs(percentage))

increase of 7.103218645948946


## 4th DF

In [297]:
inputs = df2.iloc[3001:4000, 3:4]
inputs = inputs['Tweet']

In [298]:
%%time
vectorizer.fit(inputs)
# vectorizer.get_feature_names_out()
inputs = vectorizer.transform(inputs)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



CPU times: user 2.11 s, sys: 942 µs, total: 2.11 s
Wall time: 2.16 s


In [299]:
train_preds = model.predict(inputs)
new4 = pd.Series(train_preds).value_counts()[0]
pd.Series(train_preds).value_counts()

POSITIVE    646
NEGATIVE    353
dtype: int64

In [300]:
percentage = ((new4-new3)/new4)*100

In [301]:
if percentage > 0:
  print("increase of",percentage)
else:
  print("decrease of",abs(percentage))

decrease of 39.473684210526315


## 5th DF

In [302]:
inputs = df2.iloc[4001:5000, 3:4]
inputs = inputs['Tweet']

In [303]:
%%time
vectorizer.fit(inputs)
# vectorizer.get_feature_names_out()
inputs = vectorizer.transform(inputs)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



CPU times: user 1.41 s, sys: 7 ms, total: 1.41 s
Wall time: 1.43 s


In [304]:
train_preds = model.predict(inputs)
new5 = pd.Series(train_preds).value_counts()[0]
pd.Series(train_preds).value_counts()

POSITIVE    862
NEGATIVE    137
dtype: int64

In [305]:
percentage = ((new5-new4)/new5)*100

In [306]:
if percentage > 0:
  print("increase of",percentage)
else:
  print("decrease of",abs(percentage))

increase of 25.05800464037123
